In [1]:
import torch
model = torch.hub.load("pytorch/vision", "vit_b_16")

import torchvision
from Models.transformer import VisionTransformer as vit
import Models.Conv as conv

from DataLoader import CIFAR100
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn

import utils

import numpy as np
import torch.backends.cudnn as cudnn
import random

/opt/conda/envs/torch_11_6/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using cache found in /root/.cache/torch/hub/pytorch_vision_main
/root/.cache/torch/hub/pytorch_vision_main/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")
Using cache found in /root/.cache/torch/hub/pytorch_vision_main
/root/.cache/torch/hub/pytorch_vision_main/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Using cache found in /root/.cache

In [2]:
# train_loader, test_loader = CIFAR100.get_data(22*3)
train_loader, test_loader = CIFAR100.get_data(64*3)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
layer_num = 16
teacher = vit(class_num = 100, pretrained = True, layer = layer_num)
# teacher.load_state_dict(torch.load("saved_models/vit/vit_b_16/vit_b_16.pth").module.state_dict())
teacher.load_state_dict(torch.load("saved_models/vit_b_teacher_16_88_00.pth").module.state_dict())

# student = vit(class_num = 100, pretrained = True, layer=32)
student = vit(class_num = 100, pretrained = True, layer = layer_num)
# student = vit(class_num = 100, pretrained = True, model = student)

In [4]:
device = "cuda"

teacher = teacher.to(device)
teacher = torch.nn.DataParallel(teacher, device_ids=[0, 1, 2])

student = student.to(device)
student = torch.nn.DataParallel(student, device_ids=[0, 1, 2])


In [5]:
criterion_onlylabel = lambda a,b : mse(a*b, b)
criterion_CE = nn.CrossEntropyLoss()
mse = nn.MSELoss()
softmax = torch.nn.Softmax(dim = 1)
criterion_KLD = torch.nn.KLDivLoss(reduction="batchmean")
criterion_response = lambda a,b : criterion_KLD(torch.log_softmax(a, dim=1),torch.softmax(b, dim=1))


In [6]:
S_optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)
T_optimizer = optim.SGD(teacher.parameters(), lr=0.01, momentum=0.9)
CE_loss = nn.CrossEntropyLoss()

In [7]:
S_scheduler = torch.optim.lr_scheduler.MultiStepLR(S_optimizer, milestones=[1,2,3,4,5,6,7], gamma=0.1)
T_scheduler = torch.optim.lr_scheduler.MultiStepLR(T_optimizer, milestones=[1,2,3,4,5,6,7], gamma=0.1)

In [8]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)
best_acc = 0.0
stack = 0

accs_train = []
accs_test = []

In [9]:
utils.test(teacher, test_loader,device) # student도 변하는거 확인 완료함
utils.test(student, test_loader,device) # student도 변하는거 확인 완료함


100%|██████████| 53/53 [00:27<00:00,  1.96it/s]


0 	 test acc : 0.8799999952316284


100%|██████████| 53/53 [00:19<00:00,  2.67it/s]

0 	 test acc : 0.007899999618530273


tensor(0.0079, device='cuda:0')

In [10]:
torch.cuda.empty_cache()

In [11]:
# torch.mean(fk, dim=2)에서 dim=1인 경우 잘 되지 않음

In [ ]:
student_test_accs = []

try:
    encoder_length = len(teacher.encoder.layers)
except:
    encoder_length = len(teacher.module.encoder.layers)

for epoch in range(100):
    
    print(f"lr : {S_scheduler.get_last_lr()}")

    if S_scheduler.get_last_lr()[0] < 0.000001:
        break
    
    T_correct = 0
    S_correct = 0
    all_data = 0
    
    loss_distill = []
    loss_CE = []
#     loss_response = []
    student.train()
    teacher.eval()
    for img, label in tqdm(train_loader):
        input_data = img.to(device)
        label = label.to(device)
        
        
        all_data += len(input_data)
        input_lrp = utils.get_LRP_img(input_data, label, teacher, criterion_CE, T_optimizer, mean=1.5, std = 0.1, mult=0.4).cuda()
        
        S_optimizer.zero_grad()
        T_optimizer.zero_grad()

        layer = random.randint(0,  3+encoder_length)
        input_data, fk = student(input_data,layer)
        input_lrp, fk_lrp = teacher(input_lrp,layer)
        
        if layer > 0:
            fk = torch.mean(fk, dim=2)
            fk_lrp = torch.mean(fk_lrp, dim=2)
            
        distill_loss = mse(fk, fk_lrp)
#         distill_loss = (mse(torch.mean(fk, dim=2), torch.mean(fk_lrp, dim=2)) + mse(torch.mean(fk, dim=1), torch.mean(fk_lrp, dim=1))) / 2
                    
        CE_loss = criterion_CE(input_data, label)
        
        response_loss = criterion_response(input_data, input_lrp)
        
        
        T_correct += sum(label == torch.argmax(input_lrp, dim=1))
        S_correct += sum(label == torch.argmax(input_data, dim=1))
        
        loss_CE.append(CE_loss.item())
        loss_distill.append(distill_loss.item())
#         loss_response.append(response_loss.item())
        
        loss = (distill_loss * 2 + CE_loss) / 2
        loss.backward()
        S_optimizer.step()

    print("distill loss : ", sum(loss_distill) / len(loss_distill))
    print("general loss : ", sum(loss_CE) / len(loss_CE))
#     print("response loss : ", sum(loss_response) / len(loss_response))
    
    print(f"Teacher acc: {T_correct / all_data}")
    print(f"Student acc: {S_correct / all_data}")

    test_acc = utils.test(student, test_loader,device, epoch) # student도 변하는거 확인 완료함
    
    if test_acc > best_acc + 0.01:
        stack = 0
        best_acc = test_acc
        
    else:
        stack+=1
    
    if stack > 3:
        S_scheduler.step()
        stack = 0
        
    student_test_accs.append(test_acc.item())
    print("=" * 100)

lr : [0.01]


100%|██████████| 261/261 [10:48<00:00,  2.49s/it]


distill loss :  0.001990463047101238
general loss :  2.1608902339277596
Teacher acc: 0.9528599977493286
Student acc: 0.5006600022315979


100%|██████████| 53/53 [00:20<00:00,  2.65it/s]


0 	 test acc : 0.7795999646186829
lr : [0.01]


...

100%|██████████| 53/53 [00:20<00:00,  2.64it/s]


25 	 test acc : 0.8863999843597412
lr : [1.0000000000000002e-06]


100%|██████████| 261/261 [10:44<00:00,  2.47s/it]


distill loss :  0.0011077557748637944
general loss :  0.498379146344817
Teacher acc: 0.9542199969291687
Student acc: 0.8622199892997742


 91%|█████████ | 48/53 [00:18<00:01,  2.55it/s]

In [13]:
# 99 	 test acc : 0.8797000050544739
utils.test(teacher, test_loader,device, epoch) # student도 변하는거 확인 완료함
utils.test(student, test_loader,device, epoch) # student도 변하는거 확인 완료함


100%|██████████| 53/53 [00:19<00:00,  2.66it/s]


29 	 test acc : 0.8799999952316284


100%|██████████| 53/53 [00:19<00:00,  2.66it/s]

29 	 test acc : 0.8904999494552612


tensor(0.8905, device='cuda:0')

In [15]:
saved_folder = "saved_models/vit/vit_b_16"

In [16]:
torch.save(student, f"{saved_folder}/student_vit_b_16.pth")

In [ ]:
with open(saved_folder + "/student_vit_l_16.json", "w") as f:
    json.dump({"student_test_accs" : student_test_accs}, f)